In [6]:
import pandas as pd
import numpy as np
import random
from sympy import isprime, nextprime
import os
import pickle
import random
os.chdir('Resources/')

In [7]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [8]:
def string_to_int(message):
    return int.from_bytes(message.encode('utf-8'), byteorder='big')

def int_to_string(value):
    byte_length = (value.bit_length() + 7) // 8 
    return value.to_bytes(byte_length, byteorder='big').decode('utf-8', errors='ignore')

def float_to_string(message):
    return str(message)

def string_to_float(message):
    return float(message)

In [9]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [10]:
def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1

    if m == 1:
        return 0

    while a > 1:
        q = a // m
        temp = m

        m = a % m
        a = temp
        temp = y

        y = x - q * y
        x = temp

    if x < 0:
        x += origin_m
    return x

In [11]:
df1 = pd.read_csv('6_1_CC_Encrypted_Data_Part.csv')
df1

,Age,Sex,ChestPainType,RestingBloodPressure,Cholesterol
0,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...
1,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...
2,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...
3,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...
4,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...
...,...,...,...,...,...
4039,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...
4040,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...
4041,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...
4042,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...


In [12]:
df2 = pd.read_csv('6_2_CC_Encrypted_Data_Part.csv')
df2

,FastingBloodSugar,RestingECG,MaximumHeartRate,ExerciseAngina,Oldpeak
0,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...
1,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...
2,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...
3,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...
4,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...
...,...,...,...,...,...
4039,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...
4040,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...
4041,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...
4042,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...


In [13]:
df3 = pd.read_csv('5_3_CC_Encrypted_Data_Part.csv')
df3

,ST_Slope,HeartDisease
0,(953373854764096036137291734717058396988185634...,0
1,(953373854764096036137291734717058396988185634...,1
2,(953373854764096036137291734717058396988185634...,0
3,(953373854764096036137291734717058396988185634...,1
4,(953373854764096036137291734717058396988185634...,0
...,...,...
4039,(953373854764096036137291734717058396988185634...,1
4040,(953373854764096036137291734717058396988185634...,1
4041,(953373854764096036137291734717058396988185634...,0
4042,(953373854764096036137291734717058396988185634...,0


In [14]:
df_combined = pd.concat([df1, df2, df3], axis='columns')
df_combined

,Age,Sex,ChestPainType,RestingBloodPressure,Cholesterol,FastingBloodSugar,RestingECG,MaximumHeartRate,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...,(953373854764096036137291734717058396988185634...,0
1,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...,(953373854764096036137291734717058396988185634...,1
2,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...,(953373854764096036137291734717058396988185634...,0
3,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...,(953373854764096036137291734717058396988185634...,1
4,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...,(953373854764096036137291734717058396988185634...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4039,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...,(953373854764096036137291734717058396988185634...,1
4040,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261

In [15]:
df_combined.to_csv('7_CC_Encrypted_Data.csv', index=False)

In [16]:
column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
    "Oldpeak", "ST_Slope", "HeartDisease"
]

mismatch_summary = {}
random_indices = random.sample(range(len(df_combined)), 500)
for i in range(len(random_indices)): print(random_indices[i] + 2)

1493
2806
1042
185
613
172
2054
3636
1187
2097
779
1025
3996
356
1152
680
339
3372
2225
2629
2247
216
2435
1228
1570
1704
2561
3234
726
2604
2422
3489
2853
88
3157
2014
3147
4021
2517
2329
3502
1300
1804
1415
1231
3796
1634
3351
135
1971
2873
2275
1011
1779
1302
2836
866
860
495
3129
1837
2389
919
2683
1382
1438
3449
3830
2594
95
1145
432
267
1884
3360
419
1583
3402
1255
2122
2385
2791
3915
1282
991
3228
3093
444
3238
783
3312
1414
3102
2722
2710
2372
2718
858
13
2822
2753
845
1234
1959
2616
3618
3952
2717
2452
3892
1759
1894
1446
2657
3263
1846
568
2651
3540
2811
2740
3668
2911
3133
2131
2104
3757
1637
2639
4004
4022
1195
3424
1776
2327
1543
3322
3309
2850
677
1955
2919
2795
2100
2516
3922
1047
2844
2167
932
3293
3245
2713
633
557
58
2136
2204
143
1506
2117
4009
562
2519
1952
2237
3404
1058
2957
446
3057
3769
2004
1072
1958
1128
3978
2863
1742
2691
3828
248
2635
3456
105
1001
1769
2311
3451
3337
2447
2262
2
702
3934
1692
1299
1787
2165
705
2640
422
3192
234
1670
1996
3815
3135
4023
36

In [17]:
df = pd.read_csv('7_CC_Encrypted_Data.csv')
df_org = pd.read_csv('1_CC_Structured_Data.csv')
private_key = load_key("3_CC_Private_Key.pkl")

In [18]:
for k in range(10):
    x, p, g = private_key['x'], private_key['p'], private_key['g']
    
    messages = df.iloc[random_indices, k].values
    original_column = df_org.iloc[random_indices, k].values  

    decrypted_column = []
    mismatches = [] 

    for i, (message, original_value) in zip(random_indices, zip(messages, original_column)):
        if message is not None and message == message:
            
            c1, c2 = map(int, message.strip("()").split(","))
            cur = power(c1, x, p)
            if cur == 0:
                decrypted_column.append(None)
                continue
                
            temp = mod_inv(cur, p)
            decrypted_message = (c2 * temp) % p
                
            if k == 0:
                temp_decrypted_message = decrypted_message
            elif k in [1, 2, 5, 6, 8]: 
                temp_decrypted_message = int_to_string(decrypted_message)
            elif k in [3, 4, 7, 9]: 
                temp_decrypted_message = string_to_float(int_to_string(decrypted_message))
                
            decrypted_column.append(temp_decrypted_message)
                
            if temp_decrypted_message != original_value:
                mismatches.append(i + 2)
                print(f"dec: {temp_decrypted_message}, org: {original_value}")
        else:
            decrypted_column.append(None)

    mismatch_summary[column_names[k]] = mismatches
    print(f"Mismatches for {column_names[k]}: {mismatches}")

print("\nMismatch Summary:")
for column, mismatches in mismatch_summary.items():
    print(f"{column}: {mismatches}")


Mismatches for Age: []
Mismatches for Sex: []
Mismatches for ChestPainType: []
Mismatches for RestingBloodPressure: []
Mismatches for Cholesterol: []
Mismatches for FastingBloodSugar: []
Mismatches for RestingECG: []
Mismatches for MaximumHeartRate: []
Mismatches for ExerciseAngina: []
Mismatches for Oldpeak: []

Mismatch Summary:
Age: []
Sex: []
ChestPainType: []
RestingBloodPressure: []
Cholesterol: []
FastingBloodSugar: []
RestingECG: []
MaximumHeartRate: []
ExerciseAngina: []
Oldpeak: []
